In [1]:
import numpy as np
from scipy.optimize import brentq
from scipy import integrate
import pandas as pd

## Define classes of frailties

In [2]:
class GammaFrailty:
    """ args
            RR: reletive risk (hazard ratio) of prior events
            rate: reletive risk (hazard ratio or risk ratio) of aging (hazard increases by a factor of rate per year)
            ref_T: period over which hazard ratio is averaged
            latency (<ref_T): if enterered hazard ratio is averaged over [latency, ref_T]
            T: maximal time (in unit) over which risk is updated
            constant: if True, HR is assumed to be estimated at the moment an event occurred. 
                     in this case, HR is constant over time (applies  to gamma, see Hougaard pp.235)
            alpha : form parameter of PVF distributions (a = 0 with gamma, see Hougaard pp. 241)
            r0: initial risks (per year) for which risks are updated
        values:  a tuple
            [0] updated risks (dataframe)
            [1] average hazard ratio over [latency, t] (dataframe)
            [2] parameters (dataframe)
            [3] propportion of zero-risk individuals (only relevant for compound Poisson)
            [4] mean of the frailties among event-free individuals (dataframe)
            [5] CV of the frailties among event-free individuals (dataframe)
            
    """
    def __init__(self, RR, rate, T = 5, a = 0, ref_T = None, r0 = np.linspace(0.01,0.08,8), 
                 latency = 0, constant = False):
        self.RR = RR
        self.rate = rate
        if not ref_T:
            ref_T = T
        self.ref_T = ref_T
        self.r0 = r0
        self.tau = np.arange(T+1)        
        self.constant = constant
        self.latency = latency
        self.a = a
        
    def calc_lambda_ratio(self, p0, delta):  # returns lambda0/(log(rate)*thata0) given an inital risk p0
        y = -np.log(1-p0)/delta
        r = (np.exp(y)-1)/(self.rate - 1) 
        return r
    
    def calc_lambda_0(self, p0, theta_0, delta): # returns lambda0/log(rate)
        r = self.calc_lambda_ratio(p0, delta)
        lambda_0 = theta_0*r
        return lambda_0
    
    def calc_A(self, lambda_0, t):        # returns culumative hazard up to t
        At = (self.rate**t - 1)*lambda_0
        return At

    # returns HR of failure/non-failure averaged over at time t in the population with initial risk p0

    def interval_average(self, t, lambda_0, theta_0, delta, lt_end):

        At = self.calc_A(lambda_0, t)

            #   The weight at time s is the prob density of failure at s given t 
            #   which can be written as S[s,t]*lambda[s|T>t]  (Hougaard pp.235, (7.38)) 

        def weight(s):
            As = self.calc_A(lambda_0, s)
            w = self.rate**s / (theta_0 + As + At)**(delta + 1)  # As it is a weight, constant factors are dropped
            return w

        def integrand(s):   # log HR at time t of failure at time s compared to non-failure
            As = self.calc_A(lambda_0, s)
            lnhr = np.log(1 + 1/delta) + np.log(theta_0 + 2*At) - np.log(theta_0 + As + At)
            lnhr *= weight(s)
            return lnhr

        lnHR_t = integrate.quad(integrand, lt_end, t)[0]
        lnHR_t /= integrate.quad(weight, lt_end, t)[0]

        return lnHR_t    
    
    # returns HR of failure/non-failure averaged over [0, t] in the population with initial risk p0
        
    def calc_lnHR(self, t_ref, theta_0, delta, p0, latency):  # average HR up to t_ref
        
        lambda_0 = self.calc_lambda_0(p0, theta_0, delta) 
        lt_end = 0 + latency
            
        def weight_t(t):
            At = self.calc_A(lambda_0, t)
            w = 1/(theta_0 + At)**delta
            return w
        
        def interval_average(t):

            lnHR_t = self.interval_average(t, lambda_0, theta_0, delta, lt_end)
            lnHR_t *= weight_t(t)
            
            return lnHR_t
    
        lnHR = integrate.quad(interval_average, lt_end, t_ref)[0] /integrate.quad(weight_t, lt_end, t_ref)[0]  
        
        return lnHR
    
    # given p0, returns risk over the bext year of a person who has been event-free
    
    def calc_risk(self, p0, theta_0, delta): 
        
        r = self.calc_lambda_ratio(p0, delta)
        enum  = 1 + (self.rate**(self.tau) - 1)*r
        denom = 1 + (self.rate**(self.tau + 1) - 1)*r
        p = 1 - (enum/denom)**delta
           
        return p
        
    def calc_avHR(self, p0, theta_0, delta):
        t = self.tau + 1        
        lambda_0 = self.calc_lambda_0(p0, theta_0, delta) 
        lnhr = list( map(lambda y: self.interval_average(y, lambda_0, theta_0, delta, 0), t))
        return np.exp(lnhr)

    def calc_moment(self, p_0, theta_0, delta, t, order):  # order = 1: return mean, else: return CV (sd/mean) Hougaard, p 505
        lambda_0 = self.calc_lambda_0(p_0, theta_0, delta)
        At = self.calc_A(lambda_0, t)
        theta = theta_0 + At
        mu = delta * theta**(self.a - 1) 
        if order == 1:
            return mu
        return np.sqrt( (1 - self.a)/(delta * theta**self.a ) )
        
    # finds theta0 such that given RR is equal to the average HR up to reference time, ref_T, 
    # in a reference population with initial risk p0  
    
    def find_theta_0(self, p0):
                    
        def dif_lnhr(theta_0):                
            delta = theta_0**(1 - self.a)                
            ref_lnHR = self.calc_lnHR(self.ref_T, theta_0, delta, p0, self.latency)           
            return ref_lnHR - np.log(self.RR) 
        
        theta_0 = brentq(dif_lnhr, 0.02, 10)
        
        delta = theta_0**(1 - self.a)
            
        return theta_0, delta
    
    def predict(self):
        
        res = pd.DataFrame()
        hr = pd.DataFrame()
        params = []
        
        for p0 in self.r0:

            if self.constant:
                    theta_0 = 1 /(self.RR - 1) 
                    delta = theta_0
            else:

                theta_0, delta = self.find_theta_0(p0) 
            try: 
                hr_0 =  self.calc_lnHR(0.001, theta_0, delta, p0, 0)
                hr_0 =  [np.exp(hr_0)]  # initial HR 
            except:
                hr_0 =  None

            hr_k   = self.calc_avHR(p0, theta_0, delta)
            hr_0.extend(hr_k)
            res_k = self.calc_risk(p0, theta_0, delta)
            res[p0] = res_k
            
            hr[p0] = hr_0
            params.append([theta_0, delta])
            
        zero_risk = np.exp( delta * theta_0**self.a / self.a) if self.a < 0 else 0
        params = pd.DataFrame(params)
        params.columns = ["theta_0", "delta"]
        params.index = self.r0
        
        mu = list(map(lambda t: self.calc_moment(self.r0, theta_0, delta, t, 1), self.tau) )
        cv = list(map(lambda t: self.calc_moment(self.r0, theta_0, delta, t, 2), self.tau) )
        mu = pd.DataFrame(mu)
        cv = pd.DataFrame(cv)
        mu.columns = self.r0
        cv.columns = self.r0
        
        return res, hr, params, zero_risk, mu, cv

In [3]:
class PVFFrailty(GammaFrailty):
    
    """ 
    a = 0.5 corresponds to inverse gaussian 
    a < 0 corresponds to compound poisson 
    """
    def __init__(self, RR, rate, T = 5, a = 0.5, ref_T = None, r0 = np.linspace(0.01,0.08,8), latency = 0):
        super().__init__(RR = RR, rate = rate, T = T, a = a, ref_T = ref_T, r0 = r0, latency = latency)
        
    def calc_lambda_0(self, p0, theta_0, delta):  # returns lambda0/log(rate)        
        y = - self.a * np.log(1 - p0)/delta + theta_0**self.a      
        y =  y**(1/self.a) - theta_0
        y /= self.rate - 1        
        return y
    
    def calc_S(self, theta_1, theta_2, delta):  # returns survival (see Hougaard pp.505, (A.17))
        b = theta_2**self.a - theta_1**self.a
        S = np.exp( - delta * b /self.a ) 
        return S

    def interval_average(self, t, lambda_0, theta_0, delta, lt_end): 

        At = self.calc_A(lambda_0, t) 

        def weight(s):                       # see Hougaard pp.505, (A.18)
            As = self.calc_A(lambda_0, s)   
            theta_s = theta_0 + As + At
            S = self.calc_S(theta_0, theta_s, delta)
            w = S * self.rate**s * theta_s**(self.a - 1)  # As it is a weight, constant factors are dropped
            return w

        def integrand(s):   # see Hougaard pp 244 (7.60)
            As = self.calc_A(lambda_0, s)
            At = self.calc_A(lambda_0, t)
            c = theta_0 + As + At
            y = (1- self.a) * c**(-self.a) + self.a
            lnhr = np.log(y) - np.log(delta)
            lnhr *= weight(s)
            return lnhr

        lnHR_t = integrate.quad(integrand, lt_end, t)[0]
        lnHR_t /= integrate.quad(weight, lt_end, t)[0]

        return lnHR_t    
    
    def calc_lnHR(self, t_ref, theta_0, delta, p0, latency):
        
        lambda_0 = self.calc_lambda_0(p0, theta_0, delta) 
        lt_end = 0 + latency
        
        def weight_t(t):
            At = self.calc_A(lambda_0, t)
            w = self.calc_S(theta_0, theta_0 + At, delta)
            return w

        def interval_average(t):

            lnHR_t = self.interval_average(t, lambda_0, theta_0, delta, lt_end)
            lnHR_t *= weight_t(t)
            
            return lnHR_t
       
        lnHR  = integrate.quad(interval_average, lt_end, t_ref)[0] 
        lnHR /= integrate.quad(weight_t, lt_end, t_ref)[0]         
        return lnHR        
        
    def calc_risk(self, p0, theta_0, delta):
        lambda_0 = self.calc_lambda_0(p0, theta_0, delta)
        theta_1  = theta_0 + self.calc_A(lambda_0, self.tau)
        theta_2  = theta_0 + self.calc_A(lambda_0, self.tau + 1)
        S = list( map(lambda x, y: self.calc_S(x, y, delta), theta_1, theta_2) )
        p = 1 - np.array(S) 
        return p 

In [4]:
class BetaBinom(GammaFrailty): 
    """
    values: a tuple
        [0] updated risks (fataframe)
        [1] average risk ratios (fataframe)
        [1] parameters, a and precision (fataframe)
    """
  
    def __init__(self, RR, rate, T = 5, ref_T = None, r0 = np.linspace(0.01,0.08,8), constant = False, latency = 0):
        super().__init__(RR = RR, rate = rate, T = T, ref_T = ref_T, r0 = r0, latency = latency, constant = constant)

    
    def calc_interval_average_rr(self, a, latency, n, tau_0):
        
        p = np.array( [a * self.rate**k / (tau_0 + k) for k in range(latency, n + 2)] ) # k runs over indices 
        S = [1]
        for k in range(n +1 - latency):   # k runs over indices 
            S.append( S[-1] * (1 - p[k]) )
        S = np.array(S)   
        w  = p * S           
        
        def calc_rr_km(k, m): # returns RR at year m+1 of failure at yeat k+1 

            rr_km = 1 + (self.rate-1)/( np.log(self.rate) * a * self.rate **(k) )          
            rr_km *= (tau_0 + m)/(tau_0 + k)
            return rr_km
 
        rr = []
        for k in range(latency, n): # k runs over indices       
            w_k = 0
            rr_k = 0   
            
            for j in range(latency, k+1): # j runs over time
                wj = w[j - latency]

                w_k += wj
                rr_k += wj*calc_rr_km(j+1, k+1)
                
            rr_k /= w_k
            rr.append(rr_k)
        return rr
        
        
    def calc_rr(self, a, latency, n, p0):
        
        tau_0 = a/p0       
        p = np.array( [a * self.rate**k / (tau_0 + k) for k in range(latency, n + 2)] ) # k runs over indices  
        S = [1]
        for k in range(n +1 - latency):   # k runs over indices    
            S.append( S[-1] * (1 - p[k]) )
        S = np.array(S)    
        rr = self.calc_interval_average_rr(a, latency, n, tau_0)       
        RR = 0
        weight = 0   
        for k in range(n - latency): # k runs over indices    
            RR += rr[k] * S[k+1]
            weight += S[k+1]    
        RR /= weight    
        return  RR
   
    def find_a(self, p0):        
        def calibrate(a):     
            avRR = self.calc_rr(a, self.latency, self.ref_T,  p0) 
            return np.log(avRR) - np.log(self.RR )       
        a = brentq(calibrate, 0.2, 5)
        return a

    def calc_risk_constant(self, p0):
        enum = p0*self.rate**self.tau
        denom = 1 + p0*(self.RR-1)*(self.rate**self.tau -1)/(self.rate -1)
        r = enum/denom
        return r

    def calc_average_RR(self, params, presicion): 

        RR = list( map(lambda a, tau_0: self.calc_interval_average_rr( a, 0, self.tau[-1] + 1, tau_0), 
                       params, presicion ))
        RR = pd.DataFrame(RR)
        RR.index = self.r0 
        RR.columns += 1
        return RR.T
    
    def predict(self):
        if self.constant:
            res = list(map( self.calc_risk_constant, self.r0) )
            params =  [1/(self.RR-1)]*8         #  #event corresponds to delta in gamma
        else:            
            params = list(map(self.find_a, self.r0)) 
            res = list( map(lambda a, r: a * self.rate**self.tau / ((a/r) + self.tau), params, self.r0))
        presicion = params/self.r0    # a + b corresponds to theta in gamma
        df = pd.DataFrame(res).T        
        df.columns = self.r0  
        RR = self.calc_average_RR(params, presicion)
        params = pd.DataFrame({"a": params, "precision": presicion}) 
        params.index = self.r0 
        return df, RR, params

### Gamma-av

In [5]:
model = GammaFrailty(RR = 2.5, rate = 1.5**0.1, T = 5, a = 0, ref_T = 5, r0 = np.linspace(0.01,0.08,8), 
         latency = 0, constant = False) 
pred = model.predict()

In [6]:
pred[0]   # risk estimates

,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08
0,0.010000,0.020000,0.030000,0.040000,0.050000,0.060000,0.070000,0.080000
1,0.010259,0.020233,0.029950,0.039432,0.048696,0.057757,0.066627,0.075317
2,0.010521,0.020462,0.029902,0.038901,0.047506,0.055755,0.063680,0.071309
3,0.010786,0.020687,0.029856,0.038405,0.046417,0.053959,0.061086,0.067842
4,0.011052,0.020908,0.029813,0.037940,0.045417,0.052340,0.058787,0.064816
5,0.011321,0.021124,0.029771,0.037504,0.044496,0.050875,0.056736,0.062154


### Inverse Gaussian

In [7]:
model = PVFFrailty(RR = 2.5, rate = 1.5**0.1, T = 5, a = 0.5, ref_T = 5, r0 = np.linspace(0.01,0.08,8), 
         latency = 0)  
pred = model.predict()

In [8]:
pred[1]  # point-wise HR 

,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08
0,2.598709,2.693031,2.783849,2.871809,2.957407,3.041033,3.123009,3.203597
1,2.557415,2.605470,2.645639,2.679077,2.706726,2.729370,2.747670,2.762191
2,2.517652,2.527512,2.531741,2.531861,2.528971,2.523877,2.517175,2.509314
3,2.479332,2.457396,2.435416,2.414022,2.393531,2.374088,2.355741,2.338486
4,2.442358,2.393774,2.352312,2.316541,2.285396,2.258063,2.233914,2.212454
5,2.406643,2.335617,2.279486,2.233917,2.196177,2.164434,2.137406,2.114165
6,2.372110,2.282119,2.214860,2.162563,2.120746,2.086609,2.058293,2.034512


### Beta-binomial-av

In [9]:
model = BetaBinom(RR = 2.5, rate = 1.5**0.1, T = 5, ref_T = 5, latency = 0, constant = False)
pred = model.predict()

In [10]:
pred[2]   # parameters

,a,precision
0.01,0.643773,64.377260
0.02,0.659438,32.971880
0.03,0.674342,22.478073
0.04,0.688591,17.214780
0.05,0.702267,14.045337
0.06,0.715435,11.923919
0.07,0.728150,10.402140
0.08,0.740455,9.255693
